# Preliminaries
This notebook serves as a replication exercise for proposition 3.1 of the paper "A Theory of Competition and Exchange innovation: Will the market fix the market? ",  by Budish, Lee and Shim (2023). In this notebook we will try to replicate proposition 3.1, which delineates the equilibrium in which all exchanges will choose to maintain the status quo.

The structure of the notebook is as follows: First, I will implement functions for working out equilibrium spread for the continuation games in which TFs will offer liquidity and for that in which a "lone-wolf" TF will be the sole provider of liquidity (Lemma E). Secondly, using condition 3.2 which gives bounds on the ESST fees and the previous functions (from which I can calculate profits) I would test if there will be optimal deviation for liquidity provider. Finally, I will provide unit tests for the important functions involved.



In [18]:
import pandas 
import numpy as np
import jax 
import statistics as stat
import jaxopt
import jax.numpy as jnp

In [10]:
#providing a key for random variables here
key = jax.random.PRNGKey(0)

Lemma E.1: The most fundamental element of the equilibrium is the spread, which is determined by the zero profit condition for trading firms, who provides liquidity and hence determine the spread with their limit orders:
$$ \lambda_{invest} \frac{s^*_{continuous}}{2} = \left( \lambda_{public} + \lambda_{private} \right) L(s^*_{continuous}) $$
where $L()$ represents the loss at being sniped succssfully, with 
$$ L(s) = P(J > s/2)*E(J| J>s/2)$$


The below code calculates equilibrium spread, given primitives like $\lambda_{public}$, $\lambda_{private}$, and $\lambda_{invest}$:

In [69]:
#expected loss

#generate the samples from the distribution, J, of jumps in fundamental value here
#define expected loss from fundamental value changes, given spread s.
def expectedloss(J, s):
    return jnp.array(jax.vmap(lambda x: (x > s/2)*1)(J)).mean()* jax.vmap(lambda x: (x > s/2) * x)(J).mean()


#define expected profits at the spread s: expected 
def expectedprofits(J, s, p_pri, p_pub, l_inv ):
    return l_inv*s/2 - (p_pub + p_pri) * expectedloss(J,s)

#add in rootfinding procedure to find equilibrium s.
def s_star(J, p_pri, p_pub, l_inv, N):
    #we require N>=3 from competitive regulations,
    if N <= 2:
        raise ValueError("Must have no less than 3 TF")
    result = jaxopt.Bisection(lambda s: expectedprofits(J, s, p_pri, p_pub, l_inv), -1,1.)
    eqbm_s = result.run().params
    eqbm_profits = (N-2)/((N-1)*N)*expectedloss(J, eqbm_s)

    #equilibrium spread and profits
    return eqbm_s, eqbm_profits


#example for the jump distributions
J = jax.random.truncated_normal(key, -1, 1, shape = (100,))



In [113]:
s_star(J,0,0,1,3)[0]

Array(0., dtype=float32)

Now we consider lemma E.2 (the lone-wolf lemma ):
The lone-wolf, sole liquidity provider on all exchanges with in the subset $J \subseteq J'$ where $J'$ is the set of exchanges on which all TFs purchased ESST, at the spread $s_N$ such that 
$$\lambda_{invest} \frac{s_N}{2}  - (\frac{N-2}{N-1}\lambda_{public} + \lambda_{private})L(s_N) = \frac{\lambda_{public}}{N}$$
earning a profit $\Pi_{lone-wolf} = \lambda_{invest} \frac{s_N}{2}  - (\frac{N-2}{N-1}\lambda_{public} + \lambda_{private})L(s_N)$. 

Note that the main difference between the equation above which determines equilibrium spread at which lone wolf would offer liquidity and the equation in the previous section that determines the equilibrium spread at which any trading firm would offer liquidity is that there is no opportunity costs from not sniping in the expected loss from liquidity provision.


In [87]:
def Pi_lw(J,s_n, p_pri, p_pub, l_inv, N):
    return l_inv*s_n/2 - ((N-2)/(N-1)*p_pub + p_pri)*expectedloss(J,s_n)
def s_lw(J, p_pri, p_pub, l_inv, N): 

    #check number of TF
    if N <= 2:
        raise ValueError("Must have no less than 3 TF")
    result = jaxopt.Bisection(lambda s: Pi_lw(J, s, p_pri, p_pub, l_inv, N) - p_pub*expectedloss(J,s)/N, -1,1)
    s_n = result.run().params

    #returns eqbm spread and profit for lone wolf 
    return s_n, Pi_lw(J, s_n, p_pri, p_pub,l_inv, N)
def s_min(J, p_pri,p_pub, l_inv, N):
    #check number of TF
    if N <= 2:
        raise ValueError("Must have no less than 3 TF")
    

s_lw(J,0.5,0.2,0.3,3)


(Array(0.3909912, dtype=float32), Array(0.0058577, dtype=float32))

Array(False, dtype=bool)

plans for the rest of the project:

- Prop 3.1 says being a lone wolf liquidity provider is not profitable under any circumstances, given equilibrium choices of ESST fees by the exchanges  

- testing the functions

Prop 3.2 (Bounds on ESST fees):




Below we test the validity of the above function which outputs the equiilibrium spread in the basic equilibrium of the trading game. 

Given by

In [111]:
import unittest
import jax
from itertools import product


class Test(unittest.TestCase):
    def setUp(self):
        self.key = jax.random.PRNGKey(0)
        self.J = jax.random.truncated_normal(self.key, -1, 1, shape = (100,))

    
    def test_Nvalue(self):
        N = 2
        p_pri = 0.3
        p_pub = 0.2
        l_inv = 0.5
        with self.assertRaises(ValueError):
            s_star(self.J, p_pri,p_pub,l_inv,N)



    def test_general(self):
        #testing the general relationship that s_n is smaller than s_continuous 

        #initiate by sample from symmetric distribution with mean 0 and set number of TF
        J = jax.random.truncated_normal(self.key, -1, 1)
        N = 3
        # create probability simplex
        grid = product(range(0,11),repeat = 3) 
        probsimplex = [[p/10 for p in point] for point in grid if sum(point) == 10]

        #testing if s_lw < s_star for all points in the prob simplex created above
        for i in probsimplex:
            with self.subTest(i = i):
                p_pri = i[0]
                p_pub = i[1]
                l_inv = i[2]
                self.assertGreater(s_star(J, p_pri,p_pub,l_inv,N)[0], s_lw(J, p_pri, p_pub,l_inv, N)[0])


    
    

if __name__ == '__main__':
    unittest.main(argv=[''], exit=False)


.EEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEE
ERROR: test_general (__main__.Test.test_general) (i=[0.0, 0.0, 1.0])
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/Users/ztanwork/Desktop/ECON622/.conda/lib/python3.11/site-packages/jax/_src/api.py", line 1269, in _get_axis_size
    return shape[axis]
           ~~~~~^^^^^^
IndexError: tuple index out of range

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/var/folders/tx/xr6fpk513qsbs0vq1p3fn_n00000gn/T/ipykernel_97151/481679325.py", line 37, in test_general
    self.assertGreater(s_star(J, p_pri,p_pub,l_inv,N)[0], s_lw(J, p_pri, p_pub,l_inv, N)[0])
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/tx/xr6fpk513qsbs0vq1p3fn_n00000gn/T/ipykernel_97151/2718783459.py", line 19, in s_star
    eqbm_s = result.run().params
             ^^^^^^^^^^^^
  File "/Users/zta